In [51]:
import acquire
import numpy as np
import pandas as pd
from env import username, password, host
import os
from sklearn import datasets
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as stats
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings("ignore")

from pydataset import data

import sklearn.metrics as metrics
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, recall_score
from sklearn.metrics import precision_score, f1_score
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression

# Titanic Dataset

### 1. What is your baseline prediction? What is your baseline accuracy? remember: your baseline prediction for a classification problem is predicting the most prevelant class in the training dataset (the mode). When you make those predictions, what is your accuracy? This is your baseline accuracy.

In [2]:
titanic = acquire.grab_titanic_data()
titanic.head()

,Unnamed: 0,passenger_id,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,deck,embark_town,alone
0,0,0,0,3,male,22.0,1,0,7.2500,S,Third,NaN,Southampton,0
1,1,1,1,1,female,38.0,1,0,71.2833,C,First,C,Cherbourg,0
2,2,2,1,3,female,26.0,0,0,7.9250,S,Third,NaN,Southampton,1
3,3,3,1,1,female,35.0,1,0,53.1000,S,First,C,Southampton,0
4,4,4,0,3,male,35.0,0,0,8.0500,S,Third,NaN,Southampton,1


In [3]:
titanic = acquire.prep_titanic(titanic)
#titanic.rename(columns={'sex': 'isMale'}, inplace=True)
titanic.head(10)

,survived,pclass,isMale,age,sibsp,parch,fare,embark_town,alone
0,0,3,1,22.0,1,0,7.2500,1.0,0
1,1,1,0,38.0,1,0,71.2833,2.0,0
2,1,3,0,26.0,0,0,7.9250,1.0,1
3,1,1,0,35.0,1,0,53.1000,1.0,0
4,0,3,1,35.0,0,0,8.0500,1.0,1
5,0,3,1,28.0,0,0,8.4583,3.0,1
6,0,1,1,54.0,0,0,51.8625,1.0,1
7,0,3,1,2.0,3,1,21.0750,1.0,0
8,1,3,0,27.0,0,2,11.1333,1.0,0
9,1,2,0,14.0,1,0,30.0708,2.0,0


In [4]:
print(f'{titanic.age.mean()}')
print(f'{titanic.age.mode()}')
print(titanic.age.median())

29.36158249158249
0    28.0
Name: age, dtype: float64
28.0


In [5]:
titanic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   survived     891 non-null    int64  
 1   pclass       891 non-null    int64  
 2   isMale       891 non-null    object 
 3   age          891 non-null    float64
 4   sibsp        891 non-null    int64  
 5   parch        891 non-null    int64  
 6   fare         891 non-null    float64
 7   embark_town  891 non-null    float64
 8   alone        891 non-null    int64  
dtypes: float64(3), int64(5), object(1)
memory usage: 62.8+ KB


In [6]:
#titanic.age[titanic.age.isnull()] = 28
#titanic.head(10)

In [7]:
#titanic[titanic.age == 'NaN'] = 28

In [8]:
titanic.embark_town.value_counts()

1.0    646
2.0    168
3.0     77
Name: embark_town, dtype: int64

In [9]:
#titanic.embark_town[titanic.embark_town.isnull()] = 1.0

In [10]:
#titanic.isMale[titanic.isMale == 'male'] = 1
#titanic.isMale[titanic.isMale == 'female'] = 0
#titanic.head(10)

In [11]:
t_train, t_validate, t_test = acquire.split(titanic)
t_train.shape, t_validate.shape, t_test.shape

((534, 9), (178, 9), (179, 9))

In [12]:
(t_train.survived == 1).mean()

0.40074906367041196

## What is your baseline prediction?  What is your baseline accuracy? 

In [13]:
t_train['baseline'] = 0

In [14]:
t_train.baseline.value_counts()

0    534
Name: baseline, dtype: int64

In [15]:
#t_train = t_train.drop(columns='Unnamed: 0')
t_train.head()

,survived,pclass,isMale,age,sibsp,parch,fare,embark_town,alone,baseline
304,0,3,1,28.0,0,0,8.0500,1.0,1,0
718,0,3,1,28.0,0,0,15.5000,3.0,1,0
102,0,1,1,21.0,0,1,77.2875,1.0,0,0
737,1,1,1,35.0,0,0,512.3292,2.0,1,0
269,1,1,0,35.0,0,0,135.6333,1.0,1,0


In [47]:
print(f'Baseline accuracy: {metrics.accuracy_score(t_train.survived, t_train.baseline):.3f}, This is predicting for non-survival')

Baseline accuracy: 0.599, This is predicting for non-survival


### 2. Fit the decision tree classifier to your training sample and transform (i.e. make predictions on the training sample)

In [17]:
#t_train.sex[t_train.sex == 'male'] = 1
#t_train.sex[t_train.sex == 'female'] = 0
#t_train[t_train.sex == 'female'] = 0
#t_train.rename(columns={'sex': 'isMale'})
t_train.head()

,survived,pclass,isMale,age,sibsp,parch,fare,embark_town,alone,baseline
304,0,3,1,28.0,0,0,8.0500,1.0,1,0
718,0,3,1,28.0,0,0,15.5000,3.0,1,0
102,0,1,1,21.0,0,1,77.2875,1.0,0,0
737,1,1,1,35.0,0,0,512.3292,2.0,1,0
269,1,1,0,35.0,0,0,135.6333,1.0,1,0


In [18]:
x_tr = t_train.drop(columns='survived')
y_tr = t_train.survived

x_val = t_validate.drop(columns='survived')
y_val = t_validate.survived

x_test = t_test.drop(columns='survived')
y_test = t_test.survived

x_tr.head()

,pclass,isMale,age,sibsp,parch,fare,embark_town,alone,baseline
304,3,1,28.0,0,0,8.0500,1.0,1,0
718,3,1,28.0,0,0,15.5000,3.0,1,0
102,1,1,21.0,0,1,77.2875,1.0,0,0
737,1,1,35.0,0,0,512.3292,2.0,1,0
269,1,0,35.0,0,0,135.6333,1.0,1,0


In [19]:
x_tr.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 534 entries, 304 to 133
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   pclass       534 non-null    int64  
 1   isMale       534 non-null    object 
 2   age          534 non-null    float64
 3   sibsp        534 non-null    int64  
 4   parch        534 non-null    int64  
 5   fare         534 non-null    float64
 6   embark_town  534 non-null    float64
 7   alone        534 non-null    int64  
 8   baseline     534 non-null    int64  
dtypes: float64(3), int64(5), object(1)
memory usage: 41.7+ KB


In [20]:
t_classf = DecisionTreeClassifier()
t_classf = t_classf.fit(x_tr, y_tr)

y_predict = t_classf.predict(x_tr)
y_predict_prob = t_classf.predict_proba(x_tr)

In [21]:
t_classf.fit(x_tr, y_tr)

DecisionTreeClassifier()

### 3. Evaluate your in-sample results using the model score, confusion matrix, and classification report.

In [22]:
print(f'Model score: {t_classf.score(x_tr, y_tr):.3f}')

Model score: 0.987


In [23]:
cm = confusion_matrix(y_tr, y_predict)
pd.DataFrame(cm)

,0,1
0,319,1
1,6,208


In [24]:
pd.DataFrame(classification_report(y_tr, y_predict, output_dict=True))

,0,1,accuracy,macro avg,weighted avg
precision,0.981538,0.995215,0.986891,0.988377,0.987019
recall,0.996875,0.971963,0.986891,0.984419,0.986891
f1-score,0.989147,0.983452,0.986891,0.986299,0.986865
support,320.000000,214.000000,0.986891,534.000000,534.000000


### 4. Compute: Accuracy, true positive rate, false positive rate, true negative rate, false negative rate, precision, recall, f1-score, and support.

##### Accuracy:

In [25]:
accuracy_score(y_tr, y_predict)

0.9868913857677902

##### TP, TN, FP, FN rates: 

In [26]:
TP = cm[0,0]
TN = cm[1,1]
FP = cm[0,1]
FN = cm[1,0]
print(f' True Positive rate: {TP/(TP+TN+FP+FN)}')
print(f'False Positive rate: {FP/(TP+TN+FP+FN)}')
print(f' True Negative rate: {TN/(TP+TN+FP+FN)}')
print(f'False Negative rate: {FN/(TP+TN+FP+FN)}')

 True Positive rate: 0.5973782771535581
False Positive rate: 0.0018726591760299626
 True Negative rate: 0.3895131086142322
False Negative rate: 0.011235955056179775


##### Precision, Recall, f1-score and support:

In [27]:
print(f'Precision: {precision_score(y_tr, y_predict)}')
print(f'   Recall: {recall_score(y_tr, y_predict)}')
print(f'       F1: {f1_score(y_tr, y_predict)}')
print(f'  Support: NotSurvived: {TP+FP}')
print(f'           Survived: {TN+FN}')

Precision: 0.9952153110047847
   Recall: 0.9719626168224299
       F1: 0.9834515366430261
  Support: NotSurvived: 320
           Survived: 214


### 5. Run through steps 2-4 using a different max_depth value.

This is a different Decision Tree Classifier because of the max_depth value being changed

In [28]:
t_classf2 = DecisionTreeClassifier(max_depth=4)
t_classf2 = t_classf2.fit(x_tr, y_tr)

y_predict2 = t_classf2.predict(x_tr)
y_predict_prob2 = t_classf2.predict_proba(x_tr)

In [29]:
print(f'Model score: {t_classf2.score(x_tr, y_tr)*100}')

Model score: 82.77153558052434


In [30]:
cm2 = confusion_matrix(y_tr, y_predict2)
pd.DataFrame(cm2)

,0,1
0,313,7
1,85,129


In [31]:
pd.DataFrame(classification_report(y_tr, y_predict2, output_dict=True))

,0,1,accuracy,macro avg,weighted avg
precision,0.786432,0.948529,0.827715,0.867481,0.851392
recall,0.978125,0.602804,0.827715,0.790464,0.827715
f1-score,0.871866,0.737143,0.827715,0.804505,0.817876
support,320.000000,214.000000,0.827715,534.000000,534.000000


In [32]:
print(f'Accuracy: {accuracy_score(y_tr, y_predict2)*100}')

Accuracy: 82.77153558052434


In [33]:
TP2 = cm2[0,0]
TN2 = cm2[1,1]
FP2 = cm2[0,1]
FN2 = cm2[1,0]
print('TP, TN, FP, FN rates:')
print()
print(f'TP - True Positive rate:  {TP2/(TP2+TN2+FP2+FN2):.3f}')
print(f'FP - False Positive rate: {FP2/(TP2+TN2+FP2+FN2):.3f}')
print(f'TN - True Negative rate:  {TN2/(TP2+TN2+FP2+FN2):.3f}')
print(f'FN - False Negative rate: {FN2/(TP2+TN2+FP2+FN2):.3f}')

TP, TN, FP, FN rates:

TP - True Positive rate:  0.586
FP - False Positive rate: 0.013
TN - True Negative rate:  0.242
FN - False Negative rate: 0.159


In [34]:
print('Model 1 -')
print()
print(f'Precision: {precision_score(y_tr, y_predict2):.3f}%')
print()
print(f'   Recall: {recall_score(y_tr, y_predict2):.3f}%')
print()
print(f'       F1: {f1_score(y_tr, y_predict2):.3f}%')
print()
print(f'  Support: NotSurvived: {TP2+FP2}')
print(f'           Survived:    {TN2+FN2}')

Model 1 -

Precision: 0.949%

   Recall: 0.603%

       F1: 0.737%

  Support: NotSurvived: 320
           Survived:    214


In [39]:
print('Model 2 -')
print()
print(f'Precision: {precision_score(y_tr, y_predict):.3f}%')
print()
print(f'   Recall: {recall_score(y_tr, y_predict):.3f}%')
print()
print(f'       F1: {f1_score(y_tr, y_predict):.3f}%')
print()
print(f'  Support: ')
print(f' NotSurvived: {TP+FP}')
print(f'    Survived: {TN+FN}')

Model 2 -

Precision: 0.995%

   Recall: 0.972%

       F1: 0.983%

  Support: 
 NotSurvived: 320
    Survived: 214


### 6. Which model performs better on your in-sample data?

Model -

### 7. Which model performs best on your out-of-sample data, the validate set?

In [36]:
#t_classf = DecisionTreeClassifier()
#t_classf = t_classf.fit(x_tr, y_tr)

#y_predict = t_classf.predict(x_tr)
#y_predict_prob = t_classf.predict_proba(x_tr)
val_class
y_predict = t_classf.predict(x_val)
y_predict2 = t_classf2.predict(x_val)

print(classification_report(y_val, y_predict))
print(classification_report(y_val, y_predict2))

ValueError: X has 8 features, but DecisionTreeClassifier is expecting 9 features as input.

### 1. Work through these same exercises using the Telco dataset.

In [ ]:
telco = acquire.grab_telco_data()

In [ ]:
telco = acquire.prep_telco(telco)
telco.head()

In [ ]:
tel_train, tel_validate, tel_test = acquire.split(telco)
tel_train.shape, tel_validate.shape, tel_test.shape

### 2. Experiment with this model on other datasets with a higher number of output classes.

### Iris:

In [ ]:
iris = acquire.grab_iris_data()
iris.head()

In [ ]:
iris = acquire.prep_iris(iris)
iris.head()

In [ ]:
#iris.drop(columns={'species_id', 'Unnamed: 0'})

In [ ]:
i_train, i_validate, i_test = acquire.split(iris)
i_train.shape, i_validate.shape, i_test.shape

In [ ]:
# find the baseline and the baseline accuracy
baseline = i_train.species.value_counts().idxmax()
print('The most common value (our baseline) is:', baseline)
print('The rate of occurance (our baseline accuracy) is:', len(i_train[i_train.species == baseline]) / len(i_train) * 100)

In [ ]:
# split each of our datasets into X and y 
x_train = i_train.drop(columns=('species'))
y_train = i_train.species

x_validate = i_validate.drop(columns=('species'))
y_validate = i_validate.species

x_test = i_test.drop(columns='species')
y_test = i_test.species

In [ ]:
# create and fit the models
clf1 = DecisionTreeClassifier(max_depth=8)
clf2 = DecisionTreeClassifier(max_depth=4)

clf1 = clf1.fit(X_train, y_train)
clf2 = clf2.fit(X_train, y_train)

y_pred1 = clf1.predict(X_train)
y_pred2 = clf2.predict(X_train)

y_pred_prob1 = clf1.predict_proba(X_train)
y_pred_prob2 = clf2.predict_proba(X_train)

In [ ]:
# model score
print(f'Model 1 training score: {clf1.score(X_tr, y_tr):.2%}')
print(f'Model 2 training score: {clf2.score(X_tr, y_tr):.2%}')

In [ ]:
# confusion matrix
cm1 = confusion_matrix(y_tr, y_predict1)
cm2 = confusion_matrix(y_tr, y_predict2)

print(pd.DataFrame(cm1))
print('')

print(pd.DataFrame(cm2))

In [ ]:
# classification report
print(classification_report(y_train, y_pred1))
print(classification_report(y_train, y_pred2))

## ...and comparing with validation data

In [ ]:
# classification report
y_predict1 = clf1.predict(X_validate)
y_predict2 = clf2.predict(X_validate)

print(classification_report(y_val, y_predict1))

print(classification_report(y_val, y_predict2))

# Experiment with this model on other datasets with a higher number of output classes.


# --------------------------------------------------------------------------

# Lesson: Random Forest

## 1. Fit the Random Forest classifier to your training sample and transform (i.e. make predictions on the training sample) setting the random_state accordingly and setting min_samples_leaf = 1 and max_depth = 10.

In [ ]:
titanic2 = acquire.grab_titanic_data()

In [ ]:
titanic2 = acquire.prep_titanic(titanic2)
titanic2.head()

In [ ]:
titanic2.rename(columns={'sex': 'isMale'}, inplace=True)
titanic2.head(3)

In [ ]:
#type(titanic2)

In [ ]:
titanic2.isMale[titanic2.isMale == 'male'] = 1
titanic2.isMale[titanic2.isMale == 'female'] = 0
titanic2.head(3)

In [ ]:
titanic2.age[titanic2.age.isnull()] = 28
titanic2.embark_town[titanic2.embark_town.isnull()] = 1.0

In [ ]:
titanic2.info()

In [ ]:
my_forest = RandomForestClassfifier(max_depth=10, random_state=248)

## 2. Evaluate your results using the model score, confusion matrix, and classification report.

## 3. Print and clearly label the following: Accuracy, true positive rate, false positive rate, true negative rate, false negative rate, precision, recall, f1-score, and support.

## 4. Run through steps increasing your min_samples_leaf and decreasing your max_depth.

## 5. What are the differences in the evaluation metrics? Which performs better on your in-sample data? Why?

# --------------------------------------------------------------------------

# Lesson: KNN

## 1. Fit a K-Nearest Neighbors classifier to your training sample and transform (i.e. make predictions on the training sample)

In [ ]:
x_tr.head()

weights = ['uniform', 'density']

knn = KNeighborsClassifier(n_neighbors=5, weights='uniform')

##### Creating the KNN object

In [ ]:
knn = KNeighborsClassifier(n_neighbors=5, weights='uniform')

##### Fit the model (don't forget to save back into the variable)

In [ ]:
knn = knn.fit(x_tr, y_tr)
knn

##### I'm classifying by estimated survival
##### This produces an array of prediction for the training set

In [ ]:
y_predict = knn.predict(x_tr)

##### Estimate the probability of survival, using the training data.

In [ ]:
y_pred_proba = knn.predict_proba(x_tr)
y_pred_proba

##### Compute Accuracy (Training Data)

In [ ]:
print('KNN classifier Accuracy: {:.2f}'.format(knn.score(x_tr, y_tr)))

## 2. Evaluate your results using the model score, confusion matrix, and classification report.

##### Confusion Matrix (Training Data)

In [ ]:
confus = pd.DataFrame(confusion_matrix(y_tr, y_predict))
confus

##### Classification Report (Training Data)

In [ ]:
pd.DataFrame(classification_report(y_tr, y_predict, output_dict=True))

##### Model Score

In [ ]:
print(f'Model Score: {knn.score(x_tr, y_tr):.3f}')

## 3. Print and clearly label the following: Accuracy, true positive rate, false positive rate, true negative rate, false negative rate, precision, recall, f1-score, and support.

In [ ]:
TP = confus.iloc[0,0]
TN = confus.iloc[1,1]
FP = confus.iloc[0,1]
FN = confus.iloc[1,0]
print('TP, TN, FP, FN rates:')
print()
print(f'TP - True Positive rate:  {TP/(TP+TN+FP+FN):.3f}')
print(f'FP - False Positive rate: {FP/(TP+TN+FP+FN):.3f}')
print(f'TN - True Negative rate:  {TN/(TP+TN+FP+FN):.3f}')
print(f'FN - False Negative rate: {FN/(TP+TN+FP+FN):.3f}')

In [ ]:
print(f'Precision: {precision_score(y_tr, y_predict):.3f}%')
print()
print(f'   Recall: {recall_score(y_tr, y_predict):.3f}%')
print()
print(f'       F1: {f1_score(y_tr, y_predict):.3f}%')
print()
print(f'  Support: NotSurvived: {TP+FP}')
print(f'           Survived:    {TN+FN}')

## 4. Run through steps 2-4 setting k to 10

In [ ]:
knn10 = KNeighborsClassifier(n_neighbors=10, weights='uniform')
knn10

In [ ]:
knn10 = knn10.fit(x_tr, y_tr)
knn10

In [ ]:
y_predict10 = knn10.predict(x_tr)

In [ ]:
y_pred_proba10 = knn10.predict_proba(x_tr)
y_pred_proba10

In [ ]:
confus10 = pd.DataFrame(confusion_matrix(y_tr, y_predict10))
confus10

In [ ]:
pd.DataFrame(classification_report(y_tr, y_predict10, output_dict=True))

In [ ]:
#print(f'KNN(10) Classifier Accuracy: {accuracy_score(x_tr, y_tr)}')
print(f'Model Score: {knn10.score(x_tr, y_tr):.3f}')

In [ ]:
TP = confus10.iloc[0,0]
TN = confus10.iloc[1,1]
FP = confus10.iloc[0,1]
FN = confus10.iloc[1,0]
print('TP, TN, FP, FN rates:')
print()
print(f'TP({TP})- True Positive rate:  {TP/(TP+TN+FP+FN):.3f}')
print(f'FP({FP})- False Positive rate: {FP/(TP+TN+FP+FN):.3f}')
print(f'TN({TN})- True Negative rate:  {TN/(TP+TN+FP+FN):.3f}')
print(f'FN({FN})- False Negative rate: {FN/(TP+TN+FP+FN):.3f}')

In [ ]:
print(f'TP:{TP}')
print(f'FP:{FP}')
print(f'TN:{TN}')
print(f'FN:{FN}')
print()
print(f'Accuracy:  {(TP+TN)/(TP+TN+FP+FN):.3f}%')
print()
print(f'Precision: {precision_score(y_tr, y_predict10):.3f}%')
print()
print(f'   Recall: {recall_score(y_tr, y_predict10):.3f}%')
print()
print(f'       F1: {f1_score(y_tr, y_predict10):.3f}%')
print()
print(f'  Support: NotSurvived: {TP+FP}')
print(f'           Survived:    {TN+FN}')

## 5. Run through setps 2-4 setting k to 20

In [ ]:
knn20 = KNeighborsClassifier(n_neighbors=20, weights='uniform')

In [ ]:
knn20 = knn20.fit(x_tr, y_tr)
knn20

In [ ]:
y_predict20 = knn20.predict(x_tr)

In [ ]:
y_pred_proba20 = knn20.predict_proba(x_tr)
y_pred_proba20

In [ ]:
confus20 = pd.DataFrame(confusion_matrix(y_tr, y_predict20))
confus20

In [ ]:
pd.DataFrame(classification_report(y_tr, y_predict20, output_dict=True))

In [ ]:
#print('KNN(20) classifier Accuracy: {:.2f}'.format(knn20.score(x_tr, y_tr)))
print(f'Model Score: {knn20.score(x_tr, y_tr):.3f}')

In [ ]:
TP = confus20.iloc[0,0]
TN = confus20.iloc[1,1]
FP = confus20.iloc[0,1]
FN = confus20.iloc[1,0]
print('TP, TN, FP, FN rates:')
print()
print(f'TP({TP}) - True Positive rate:  {TP/(TP+TN+FP+FN):.3f}')
print(f'FP({FP}) - False Positive rate: {FP/(TP+TN+FP+FN):.3f}')
print(f'TN({TN}) - True Negative rate:  {TN/(TP+TN+FP+FN):.3f}')
print(f'FN({FN}) - False Negative rate: {FN/(TP+TN+FP+FN):.3f}')

In [ ]:
print(f'TP:{TP}')
print(f'FP:{FP}')
print(f'TN:{TN}')
print(f'FN:{FN}')
print()
print(f'Accuracy:  {(TP+TN)/(TP+TN+FP+FN):.3f}%')
print()
print(f'Precision: {precision_score(y_tr, y_predict):.3f}%')
print()
print(f'   Recall: {recall_score(y_tr, y_predict):.3f}%')
print()
print(f'       F1: {f1_score(y_tr, y_predict):.3f}%')
print()
print(f'  Support: NotSurvived: {TP+FP}')
print(f'           Survived:    {TN+FN}')

## 6. What are the differences in the evaluation metrics? Which performs better on your in-sample data? Why?

## 7. Which model performs best on our out-of-sample data from validate?

# ----------------------------------------------------------

# __Lesson: Logistic Regression__

### Instructions:
In these exercises, we'll continue working with the titanic dataset and building logistic regression models. Throughout this exercise, be sure you are training, evaluation, and comparing models on the train and validate datasets. The test dataset should only be used for your final model.

For all of the models you create, choose a threshold that optimizes for accuracy.

***

## Model 1
### Make the model
**Create the object**

logit = LogisticRegression(C=1, class_weight={0:1, 1:99}, random_state=123, intercept_scaling=1, solver='lbfgs')

**Fit the model**Fit the random forest algorithm to the training data.

logit.fit(X_train, y_train)

LogisticRegression(C=1, class_weight={0: 1, 1: 99}, random_state=123)

**Feature Importance**Evaluate importance, or weight, of each feature, using the coefficients.

Evaluate the intercept of the model.

print('Coefficient: \n', logit.coef_)
print('Intercept: \n', logit.intercept_)

**Make Predictions**Estimate whether or not the species is versicolor for each observation, using the training data.

y_pred = logit.predict(X_train)

**Estimate Probability**Estimate the probability of species being versicolor for each observation, using the training data.

y_pred_proba = logit.predict_proba(X_train)
***
### Evaluate the Model
**Compute the Accuracy**

print('Accuracy of Logistic Regression classifier on training set: {:.2f}'.format(logit.score(X_train, y_train)))
     
**Create a confusion matrix**

print(confusion_matrix(y_train, y_pred))

**Create a classification report**

print(classification_report(y_train, y_pred))
***
## Model 2
### Make the Model
**Create the object**

logit2 = LogisticRegression(C=.1, class_weight={0:1, 1:99}, random_state=123, intercept_scaling=1, solver='lbfgs')

**Fit the model:**Fit the random forest algorithm to the training data.

logit2.fit(X_train, y_train)

LogisticRegression(C=0.1, class_weight={0: 1, 1: 99}, random_state=123)

**Feature Importance:**Evaluate importance, or weight, of each feature, using the coefficients.Evaluate the intercept of the model.

print('Coefficient: \n', logit2.coef_)
print('Intercept: \n', logit2.intercept_)

**Make Predictions:**Estimate whether or not the species is versicolor for each observation, using the training data.

y_pred2 = logit2.predict(X_train)

**Estimate Probability**

Estimate the probability of species being versicolor for each observation, using the training data.

y_pred_proba2 = logit2.predict_proba(X_train)
***
### Evaluate Model
**Compute the Accuracy**

print('Accuracy of Logistic Regression classifier on training set: {:.2f}'.format(logit2.score(X_train, y_train)))

**Create a confusion matrix**

print(confusion_matrix(y_train, y_pred2))

**Create a classification report**

print(classification_report(y_train, y_pred2))

**Compute Scores**

Accuracy<br>
Precision<br>
Recall<br>
F1 Score<br>
Support

### Validate Models
### Test Models
### Visualize Models

***

## 1. Create a model that includes age in addition to fare and pclass. Does this model perform better than your baseline? 

In [40]:
titanic = acquire.grab_titanic_data()
titanic.head(3)

,Unnamed: 0,passenger_id,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,deck,embark_town,alone
0,0,0,0,3,male,22.0,1,0,7.2500,S,Third,NaN,Southampton,0
1,1,1,1,1,female,38.0,1,0,71.2833,C,First,C,Cherbourg,0
2,2,2,1,3,female,26.0,0,0,7.9250,S,Third,NaN,Southampton,1


In [42]:
titanic = acquire.prep_titanic(titanic)
titanic.head(3)

,survived,pclass,isMale,age,sibsp,parch,fare,embark_town,alone
0,0,3,1,22.0,1,0,7.2500,1.0,0
1,1,1,0,38.0,1,0,71.2833,2.0,0
2,1,3,0,26.0,0,0,7.9250,1.0,1


In [44]:
titanic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   survived     891 non-null    int64  
 1   pclass       891 non-null    int64  
 2   isMale       891 non-null    object 
 3   age          891 non-null    float64
 4   sibsp        891 non-null    int64  
 5   parch        891 non-null    int64  
 6   fare         891 non-null    float64
 7   embark_town  891 non-null    float64
 8   alone        891 non-null    int64  
dtypes: float64(3), int64(5), object(1)
memory usage: 62.8+ KB


In [45]:
titanic['baseline'] = 0

In [46]:
titanic.head()

,survived,pclass,isMale,age,sibsp,parch,fare,embark_town,alone,baseline
0,0,3,1,22.0,1,0,7.2500,1.0,0,0
1,1,1,0,38.0,1,0,71.2833,2.0,0,0
2,1,3,0,26.0,0,0,7.9250,1.0,1,0
3,1,1,0,35.0,1,0,53.1000,1.0,0,0
4,0,3,1,35.0,0,0,8.0500,1.0,1,0


In [49]:
#print(f'Baseline accuracy: {metrics.accuracy_score(titanic.survived, titanic.baseline):.3f}, This is predicting for non-survival')

In [56]:
train, validate, test = acquire.split(titanic)
train.shape, validate.shape, test.shape

((534, 10), (178, 10), (179, 10))

In [58]:
X_train = train.drop(columns='survived')
y_train = train.survived
X_validate = validate.drop(columns='survived')
y_validate = validate.survived
X_test = test.drop(columns='survived')
y_test = test.survived

##### Define Logistic Regression Model

In [59]:
logit = LogisticRegression()#C=.1, class_weight={0:1, 1:99}, random_state=248, intercept_scaling=1, solver='lbfgs')

##### Fit the model to the train data

In [60]:
logit.fit(X_train, y_train)

LogisticRegression()

##### Make predictions using the model

In [63]:
y_pred = logit.predict(X_train)
y_pred[:20]

array([0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1])

In [69]:
y_pred_proba = logit.predict_proba(X_train)
y_pred_proba = pd.DataFrame(y_pred_proba, columns=['NotSurvive', 'Survive'])
y_pred_proba.head(10)

,NotSurvive,Survive
0,0.888720,0.111280
1,0.872770,0.127230
2,0.277540,0.722460
3,0.234243,0.765757
4,0.093453,0.906547
5,0.908249,0.091751
6,0.370202,0.629798
7,0.590165,0.409835
8,0.673378,0.326622
9,0.980333,0.019667


### With just pclass, age, fare

In [80]:
titanic.head(3)

,survived,pclass,isMale,age,sibsp,parch,fare,embark_town,alone,baseline
0,0,3,1,22.0,1,0,7.2500,1.0,0,0
1,1,1,0,38.0,1,0,71.2833,2.0,0,0
2,1,3,0,26.0,0,0,7.9250,1.0,1,0


In [81]:
dt = titanic.drop(columns={'embark_town', 'alone', 'sibsp', 'parch',
                  'isMale'})
dt.head(3)

,survived,pclass,age,fare,baseline
0,0,3,22.0,7.2500,0
1,1,1,38.0,71.2833,0
2,1,3,26.0,7.9250,0


In [82]:
train, validate, test = acquire.split(dt)
train.shape, validate.shape, test.shape

((534, 5), (178, 5), (179, 5))

In [83]:
X_train = train.drop(columns='survived')
y_train = train.survived
X_validate = validate.drop(columns='survived')
y_validate = validate.survived
X_test = test.drop(columns='survived')
y_test = test.survived

In [84]:
logit.fit(X_train, y_train)

LogisticRegression()

In [85]:
y_pred = logit.predict(X_train)
y_pred[:20]

array([0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1])

In [86]:
y_pred_proba = logit.predict_proba(X_train)
y_pred_proba = pd.DataFrame(y_pred_proba, columns=['NotSurvive', 'Survive'])
y_pred_proba.head(10)

,NotSurvive,Survive
0,0.762711,0.237289
1,0.756428,0.243572
2,0.224624,0.775376
3,0.061889,0.938111
4,0.273070,0.726930
5,0.805457,0.194543
6,0.186862,0.813138
7,0.815493,0.184507
8,0.447797,0.552203
9,0.567083,0.432917


## 2. Include sex in your model as well. Note that you'll need to encode or create a dummy variable of this feature before including it in a model.

In [87]:
titanic.head(3)

,survived,pclass,isMale,age,sibsp,parch,fare,embark_town,alone,baseline
0,0,3,1,22.0,1,0,7.2500,1.0,0,0
1,1,1,0,38.0,1,0,71.2833,2.0,0,0
2,1,3,0,26.0,0,0,7.9250,1.0,1,0


In [88]:
withsex = titanic.drop(columns={'embark_town', 'alone', 'sibsp', 'parch'})
withsex.head(3)

,survived,pclass,isMale,age,fare,baseline
0,0,3,1,22.0,7.2500,0
1,1,1,0,38.0,71.2833,0
2,1,3,0,26.0,7.9250,0


In [89]:
train, validate, test = acquire.split(withsex)
train.shape, validate.shape, test.shape

((534, 6), (178, 6), (179, 6))

In [90]:
X_train = train.drop(columns='survived')
y_train = train.survived
X_validate = validate.drop(columns='survived')
y_validate = validate.survived
X_test = test.drop(columns='survived')
y_test = test.survived

In [91]:
logit.fit(X_train, y_train)

LogisticRegression()

In [92]:
y_pred = logit.predict(X_train)
y_pred[:20]

array([0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1])

In [93]:
y_pred_proba = logit.predict_proba(X_train)
y_pred_proba = pd.DataFrame(y_pred_proba, columns=['NotSurvive', 'Survive'])
y_pred_proba.head(10)

,NotSurvive,Survive
0,0.897273,0.102727
1,0.895999,0.104001
2,0.407574,0.592426
3,0.341245,0.658755
4,0.089263,0.910737
5,0.917676,0.082324
6,0.390648,0.609352
7,0.543937,0.456063
8,0.622098,0.377902
9,0.800811,0.199189


In [96]:
pd.DataFrame(classification_report(y_train, y_pred, output_dict=True))

,0,1,accuracy,macro avg,weighted avg
precision,0.807453,0.716981,0.771536,0.762217,0.771197
recall,0.812500,0.710280,0.771536,0.761390,0.771536
f1-score,0.809969,0.713615,0.771536,0.761792,0.771355
support,320.000000,214.000000,0.771536,534.000000,534.000000


## 3. Try out other combinations of features and models.

## 4. Use you best 3 models to predict and evaluate on your validate sample.

## 5. Choose you best model from the validation performation, and evaluate it on the test dataset. How do the performance metrics compare to validate? to train?

Bonus1 How do different strategies for handling the missing values in the age column affect model performance?

Bonus2: How do different strategies for encoding sex affect model performance?

Bonus3: scikit-learn's LogisticRegression classifier is actually applying a regularization penalty to the coefficients by default. This penalty causes the magnitude of the coefficients in the resulting model to be smaller than they otherwise would be. This value can be modified with the C hyper parameter. Small values of C correspond to a larger penalty, and large values of C correspond to a smaller penalty.
Try out the following values for C and note how the coefficients and the model's performance on both the dataset it was trained on and on the validate split are affected.
C
=
.01
,
.1
,
1
,
10
,
100
,
1000
Bonus Bonus: how does scaling the data interact with your choice of C?

<div class="alert alert-block alert-info">
<b>Tip:</b> Use blue boxes (alert-info) for tips and notes. 
If it’s a note, you don’t have to include the word “Note”.
</div>

<div class="alert alert-block alert-warning">
<b>Example:</b> Use yellow boxes for examples that are not 
inside code cells, or use for mathematical formulas if needed.
</div>

<div class="alert alert-block alert-success">
<b>Up to you:</b> Use green boxes sparingly, and only for some specific 
purpose that the other boxes can't cover. For example, if you have a lot 
of related content to link to, maybe you decide to use green boxes for 
related links from each section of a notebook.
</div>

<div class="alert alert-block alert-danger">
<b>Just don't:</b> In general, avoid the red boxes. These should only be
used for actions that might cause data loss or another major issue.
</div>

> fffffff

***

&#9719;</p>